In [1]:
!git clone https://github.com/lanzhang128/multi_agent_autoformalization.git
%cd multi_agent_autoformalization
!mkdir test_results

Cloning into 'multi_agent_autoformalization'...
remote: Enumerating objects: 148, done.
remote: Counting objects: 100% (148/148), done.
remote: Compressing objects: 100% (141/141), done.
remote: Total 148 (delta 72), reused 6 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (148/148), 5.56 MiB | 8.34 MiB/s, done.
Resolving deltas: 100% (72/72), done.
/content/multi_agent_autoformalization


In [2]:
!pip install rank_bm25

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!tar -xzf /content/drive/MyDrive/Isabelle2024_linux.tar.gz -C /content/
print('Extraction finished.')

Extraction finished.


In [5]:
import os
os.environ['ISABELLE_DIRPATH'] = os.path.abspath('/content/Isabelle2024')

import nest_asyncio
nest_asyncio.apply()

#Settings

###Prepare three examples for few-shot autoformalization

In [6]:
informal_formal_pairs = [
    [
        "Determine the value of $ab$ if $\\log_8a+\\log_4b^2=5$ and $\\log_8b+\\log_4a^2=7$. Show that it is 512.",
        "theory aime_1984_p5 imports Complex_Main\nbegin\n\ntheorem aime_1984_p5:\n  fixes a b ::real\n  assumes "
        "\"(ln a) / (ln 8) + (ln (b^2)) / (ln 4) = 5\"\n          \"(ln b) / (ln 8) + (ln (a^2)) / (ln 4) = 7\"\n "
        "       shows \"a * b = 512\"\n  sorry\n\nend\n"
        ],
    [
        "One dimension of a cube is increased by $1$, another is decreased by $1$, and the third is left "
        "unchanged. The volume of the new rectangular solid is $5$ less than that of the cube. What was the "
        "volume of the cube?\n\n$\\textbf{(A)}\\ 8 \\qquad \\textbf{(B)}\\ 27 \\qquad \\textbf{(C)}\\ 64 \\qquad "
        "\\textbf{(D)}\\ 125 \\qquad \\textbf{(E)}\\ 216$ Show that it is \\text{(D)}.",
        "theory amc12a_2009_p5 imports\n  Complex_Main\nbegin\n\ntheorem amc12a_2009_p5:\n  fixes x :: real\n  "
        "assumes h0 : \"x^3 - (x+1) * (x-1) * x = 5\"\n  shows \"x^3 = 125\"\n  sorry\n\nend"
        ],
    [
        "Solve the system of equations\n\n$|a_1 - a_2| x_2 +|a_1 - a_3| x_3 +|a_1 - a_4| x_4 = 1\\\\ |a_2 - a_1| "
        "x_1 +|a_2 - a_3| x_3 +|a_2 - a_4| x_4 = 1\\\\ |a_3 - a_1| x_1 +|a_3 - a_2| x_2 +|a_3-a_4|x_4= 1\\\\ |a_4 "
        "- a_1| x_1 +|a_4 - a_2| x_2 +|a_4 - a_3| x_3 = 1$\n\nwhere $a_1, a_2, a_3, a_4$ are four different real "
        "numbers.",
        "theory imo_1966_p5 imports\nComplex_Main\n\nbegin\n\ntheorem imo_1966_p5:\n  fixes x a :: \"nat "
        "\\<Rightarrow> real\"\n  assumes \"a 1 > a 2\" and \"a 2 > a 3\" and \"a 3 > a 4\"\n  assumes \n    h6 : "
        "\"abs (a 1 - a 2) * x 2 + abs (a 1 - a 3) * x 3 + abs (a 1 - a 4) * x 4 = 1\"\n    and h7 : \"abs (a 2 - "
        "a 1) * x 1 + abs (a 2 - a 3) * x 3 + abs (a 2 - a 4) * x 4 = 1\"\n    and h8 : \"abs (a 3 - a 1) * x 1 + "
        "abs (a 3 - a 2) * x 2 + abs (a 3 - a 4) * x 4 = 1\"\n    and h9 : \"abs (a 4 - a 1) * x 1 + abs (a 4 - a "
        "2) * x 2 + abs (a 4 - a 3) * x 3 = 1\"\n  shows \"x 2 = 0 \\<and> x 3 = 0 \\<and> x 1 = 1 / abs (a 1 - a "
        "4) \\<and> x 4 = 1 / abs (a 1 - a 4)\"\n  sorry\n\nend"
        ]
    ]

###Prepare test natural language mathematical statements

In [7]:
informal_statements = [
    "Find the value of $a_2+a_4+a_6+a_8+\\ldots+a_{98}$ if $a_1$, $a_2$, $a_3\\ldots$ is an [[arithmetic progression]] "
    "with common difference 1, and $a_1+a_2+a_3+\\ldots+a_{98}=137$. Show that it is 093.",
    "Definition of Softmax Function: Formally, the standard (unit) softmax function "
    "\\(\\sigma\\colon \\mathbb{R}^K \\to (0, 1)^K\\), where \\(K \\ge 1\\), takes a vector "
    "\\(\\mathbf{z} = (z_1, \\dotsc, z_K) \\in \\mathbb{R}^K\\) and computes each component of vector "
    "\\(\\sigma(\\mathbf{z}) \\in (0, 1)^K\\) with \\(\\sigma(\\mathbf{z})_i = \\frac{e^{z_i}}{\\sum_{j=1}^K e^{z_j}}\\,.\\)",
    "Definition of Isotropic Position: A probability distribution over vectors is said to be in isotropic"
    "position if its covariance matrix is equal to the identity matrix. Formally, let \\(D\\) be a "
    "distribution over vectors in the vector space \\(\\mathbb{R}^n\\). Then \\(D\\) is in isotropic "
    "position if, for vector \\(v\\) sampled from the distribution, \\(\\mathbb{E}\\, vv^\\mathsf{T} = "
    "\\mathrm{Id}.\\)"
]

###Set an aspect of interest

In [8]:
aspect_description='whether the formalized code involves all mathematical concepts in the natural language statement.'

### Self-Improving Multi-Agent

In [ ]:
from llm import OpenAILLM
from agent import (AutoformalizationAgent,
           HardCritiqueAgent, FormalRefinementAgent,
           SoftCritiqueAgent, InformalRefinementAgent)


def self_improving(informal_statements, informal_formal_pairs, aspect_description):
  # set formal language
  formal_language = 'Isabelle/HOL'

  gpt_4o_mini = OpenAILLM(
    api_key='api-key',
    model='gpt-4o-mini')

  # Instantiate Agents
  agent_auto = AutoformalizationAgent(
      llm=gpt_4o_mini,
      formal_language=formal_language)

  agent_hard = HardCritiqueAgent(
      formal_language=formal_language,
      file_dir='./test_results')

  agent_formal = FormalRefinementAgent(
      llm=gpt_4o_mini,
      formal_language=formal_language)

  agent_soft = SoftCritiqueAgent(
      llm=gpt_4o_mini,
      formal_language=formal_language,
      aspect_description=aspect_description)

  agent_informal = InformalRefinementAgent(
      llm=gpt_4o_mini,
      formal_language=formal_language)

  results = []
  for informal_statement in informal_statements:
      # few-shot autoformalization
      formalization, _ = agent_auto(
          informal_statement=informal_statement,
          informal_formal_pairs=informal_formal_pairs)

      # obtain hard critiques of the formalization
      correctness, error_details = agent_hard(
          formalization=formalization,
          file_prefix='test')

      # refine formalization if it is syntactically incorrect
      if correctness != 'True':
        formal_refinement, _ = agent_formal(
            informal_statement=informal_statement,
            refinement_mode='detailed',
            formalization_file='./test_results/test.thy',
            correctness=correctness,
            error_details=error_details)
      else:
        formal_refinement = formalization

      # obtain soft critiques of the formalization
      aspect_evaluation, _ = agent_soft(
          informal_statement=informal_statement,
          formalization=formalization)

      # refine formalization with soft critiques
      informal_refinement, _ = agent_informal(
          informal_statement=informal_statement,
          formalization=formalization,
          aspect_description=aspect_description,
          aspect_evaluation=aspect_evaluation)

      # collect results
      results.append({
          'formalization': formalization,
          'correctness': correctness,
          'error_details': error_details,
          'formal_refinement': formal_refinement,
          'aspect_evaluation': aspect_evaluation,
          'informal_refinement': informal_refinement
          })

  # terminate the backend theorem prover to release resources
  agent_hard.theorem_prover.terminate()
  return results

In [10]:
self_improving_results = self_improving(informal_statements, informal_formal_pairs, aspect_description)

logging file is at: ./test_results/isabelle.log
Isabelle server started with info: server "test" = 127.0.0.1:40033 (password "81575e53-4729-4029-a7b4-64c6b764ca7f")
 in 2.00s.
Isabelle server session 61a8de27-065b-4ba1-87dd-27949a67010b started in 11.56s.
Isabelle server use_theory ended in 2.38s.
Isabelle server use_theory ended in 2.27s.
Isabelle server use_theory ended in 2.26s.
Isabelle server 127.0.0.1:40033 shutdown.


In [11]:
for informal_statement, res in zip(informal_statements, self_improving_results):
  print(f'informal statement:\n{informal_statement}')
  for key in res.keys():
    print(f'{key}:\n{res[key]}')

informal statement:
Find the value of $a_2+a_4+a_6+a_8+\ldots+a_{98}$ if $a_1$, $a_2$, $a_3\ldots$ is an [[arithmetic progression]] with common difference 1, and $a_1+a_2+a_3+\ldots+a_{98}=137$. Show that it is 093.
formalization:
theory arithmetic_progression imports Complex_Main
begin

theorem arithmetic_progression:
  fixes a1 :: real
  assumes "a1 + (a1 + 1) + (a1 + 2) + ... + (a1 + 97) = 137"
  shows "a1 + (a1 + 1) + (a1 + 3) + ... + (a1 + 97) = 93"
  sorry

end

correctness:
False
error_details:
Identified error on line: 5
Error message: Unbound schematic variable: \<dots>

Identified error on line: 9
Error message: Bad context for command "sorry"\<^here> -- using reset state


formal_refinement:
theory test
imports Complex_Main
begin

theorem arithmetic_progression:
  fixes a1 :: real
  assumes "a1 + (a1 + 1) + (a1 + 2) + (a1 + 3) + ... + (a1 + 97) = 137"
  shows "a1 + (a1 + 1) + (a1 + 3) + (a1 + 5) + ... + (a1 + 97) = 93"
  sorry

end

aspect_evaluation:
Judgement: False
Explan

### Iterative Refinement Multi-Agent

In [ ]:
def iterative_refinement(informal_statements, formalizations, iterations):
  # set formal language
  formal_language = 'Isabelle/HOL'

  gpt_4o = OpenAILLM(
    api_key='api-key',
    model='gpt-4o')

  # Instantiate Agents
  agent_hard = HardCritiqueAgent(
      formal_language=formal_language,
      file_dir='./test_results')

  agent_formal = FormalRefinementAgent(
      llm=gpt_4o,
      formal_language=formal_language)

  results = []
  for informal_statement, formalization in zip(informal_statements, formalizations):
      results.append([])
      # iterations
      for i in range(iterations):
        # obtain hard critiques of the formalization
        correctness, error_details = agent_hard(
            formalization=formalization,
            file_prefix='test')
        results[-1].append({
            'formalization': formalization,
            'correctness': correctness,
            'error_details': error_details
            })
        # refine formalization if it is syntactically incorrect
        if correctness != 'True':
          formalization, _ = agent_formal(
            informal_statement=informal_statement,
            refinement_mode='detailed',
            formalization_file='./test_results/test.thy',
            correctness=correctness,
            error_details=error_details)


      results[-1].append({'formalization': formalization})

  # terminate the backend theorem prover to release resources
  agent_hard.theorem_prover.terminate()
  return results

In [13]:
formalizations = [res['formalization'] for res in self_improving_results]

In [14]:
iterative_refinement_results = iterative_refinement(informal_statements, formalizations, iterations=5)

logging file is at: ./test_results/isabelle.log
Isabelle server started with info: server "test" = 127.0.0.1:42163 (password "b37bd0da-af8b-4da2-b9c4-99c40a048fe5")
 in 2.04s.
Isabelle server session c355f33e-3e56-40a4-aae0-7ef0d406c92d started in 10.68s.
Isabelle server use_theory ended in 2.50s.
Isabelle server use_theory ended in 0.52s.
Isabelle server use_theory ended in 0.20s.
Isabelle server use_theory ended in 0.28s.
Isabelle server use_theory ended in 0.29s.
Isabelle server use_theory ended in 1.38s.
Isabelle server use_theory ended in 0.29s.
Isabelle server use_theory ended in 0.29s.
Isabelle server use_theory ended in 0.29s.
Isabelle server use_theory ended in 0.29s.
Isabelle server use_theory ended in 0.95s.
Isabelle server use_theory ended in 1.02s.
Isabelle server use_theory ended in 0.37s.
Isabelle server use_theory ended in 0.83s.
Isabelle server use_theory ended in 0.95s.
Isabelle server 127.0.0.1:42163 shutdown.


In [15]:
for informal_statement, res in zip(informal_statements, iterative_refinement_results):
  print(f'informal statement:\n{informal_statement}')
  for i in range(len(res)):
    print(f'iteration {i}:')
    for key in res[i].keys():
      print(f'{key}:\n{res[i][key]}')

informal statement:
Find the value of $a_2+a_4+a_6+a_8+\ldots+a_{98}$ if $a_1$, $a_2$, $a_3\ldots$ is an [[arithmetic progression]] with common difference 1, and $a_1+a_2+a_3+\ldots+a_{98}=137$. Show that it is 093.
iteration 0:
formalization:
theory arithmetic_progression imports Complex_Main
begin

theorem arithmetic_progression:
  fixes a1 :: real
  assumes "a1 + (a1 + 1) + (a1 + 2) + ... + (a1 + 97) = 137"
  shows "a1 + (a1 + 1) + (a1 + 3) + ... + (a1 + 97) = 93"
  sorry

end

correctness:
False
error_details:
Identified error on line: 5
Error message: Unbound schematic variable: \<dots>

Identified error on line: 9
Error message: Bad context for command "sorry"\<^here> -- using reset state


iteration 1:
formalization:
theory test
imports Complex_Main
begin

theorem arithmetic_progression:
  fixes a1 :: real
  assumes "(\<Sum>i=0..97. a1 + i) = 137"
  shows "(\<Sum>i=1..49. a1 + (2 * i - 1)) = 93"
  sorry

end

correctness:
True
error_details:

iteration 2:
formalization:
theory t